## 0 · Environment Setup

In [ ]:
# ── Install dependencies (Kaggle already has most, add extras) ──
!pip install -q ultralytics peft bitsandbytes qwen-vl-utils \
    rasterio geopandas albumentations wandb transformers accelerate --upgrade

In [ ]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# ── Set W&B API key from Kaggle Secrets or env ──
# Option 1: Kaggle Secrets (recommended)
try:
    from kaggle_secrets import UserSecretsClient
    secrets = UserSecretsClient()
    os.environ['WANDB_API_KEY'] = secrets.get_secret('WANDB_API_KEY')
    print('✓ W&B API key loaded from Kaggle Secrets')
except Exception:
    print('⚠ Set WANDB_API_KEY manually: os.environ["WANDB_API_KEY"] = "your-key"')

# ── Set SpaceNet 7 data root (auto-detects Kaggle input) ──
# Change the dataset name below to match YOUR Kaggle input dataset name
SPACENET_DATASET_NAME = 'spacenet7'  # adjust if your dataset is named differently
if os.path.exists(f'/kaggle/input/{SPACENET_DATASET_NAME}'):
    os.environ['SPACENET7_ROOT'] = f'/kaggle/input/{SPACENET_DATASET_NAME}'
    print(f'✓ SpaceNet 7 data root: /kaggle/input/{SPACENET_DATASET_NAME}')
else:
    # List available input datasets
    if os.path.exists('/kaggle/input'):
        available = os.listdir('/kaggle/input')
        print(f'Available datasets: {available}')
        print('Set os.environ["SPACENET7_ROOT"] to the correct path.')
    else:
        print('Running locally — set SPACENET7_ROOT env variable.')

print(f'Python: {sys.version}')

In [ ]:
# ── Load GeoExtract config ──
from config import CFG, IS_KAGGLE
from utils import log_vram, free_vram

print(CFG.summary())
log_vram('startup')

---
## 1 · Data Pipeline — SpaceNet 7 → YOLO Format

In [ ]:
from data_pipeline import YOLODatasetBuilder

# Build YOLO dataset from SpaceNet 7
builder = YOLODatasetBuilder()
dataset_yaml = builder.build()

# Print stats
stats = builder.get_stats()
print('\n📊 Dataset Statistics:')
for split, s in stats.items():
    print(f'  {split}: {s["images"]} images, {s["total_bboxes"]} boxes '
          f'({s["avg_bboxes_per_image"]} avg/img)')

## 2 · Synthetic QA Generation for VLM

In [ ]:
from qa_generator import SyntheticQAGenerator

qa_gen = SyntheticQAGenerator()
qa_train_path = qa_gen.generate()

# Print stats
qa_stats = qa_gen.get_stats()
print('\n📊 QA Generation Statistics:')
for split, s in qa_stats.items():
    print(f'  {split}: {s["conversations"]} conversations, '
          f'{s["total_qa_turns"]} QA turns')
    print(f'    Density distribution: {s["density_distribution"]}')

---
## 3 · Phase 1: YOLO Training (Building Detection)

**VRAM Budget:** ~4-6 GB  
Training YOLOv11-nano with auto-resume capability.

In [ ]:
from yolo_trainer import YOLOTrainer

yolo_trainer = YOLOTrainer(dataset_yaml=dataset_yaml)
best_yolo_weights = yolo_trainer.train()

print(f'\n✓ Best YOLO weights: {best_yolo_weights}')

In [ ]:
# Validate YOLO and get hard metrics
yolo_metrics = yolo_trainer.validate()
print(f'\n📊 YOLO Metrics: {yolo_metrics}')

# CRITICAL: Free YOLO from GPU before loading VLM
yolo_trainer.cleanup()

---
## 4 · Phase 2: VLM Training (Urban Reasoner)

**VRAM Budget:** ~10-12 GB  
Qwen2-VL-2B in 4-bit NF4 with LoRA (r=16, α=32).  
Cosine LR schedule with 10% warmup.  
Checkpoints every 500 steps — auto-resumes on Kaggle restart.

In [ ]:
from vlm_trainer import VLMTrainer

vlm_trainer = VLMTrainer()
adapter_path = vlm_trainer.train()

print(f'\n✓ VLM adapter saved to: {adapter_path}')

In [ ]:
# Free VLM training resources
vlm_trainer.cleanup()

---
## 5 · Evaluation — Hard Metrics for Defense

Computes F1, Precision, Recall for:
- **YOLO** — Building detection (IoU-based)
- **VLM** — Density classification (confusion matrix)

In [ ]:
from evaluate import GeoExtractEvaluator
from inference import GeoExtractPipeline

# Load the inference pipeline for VLM evaluation
pipeline = GeoExtractPipeline()
pipeline.load()

# Run full evaluation
evaluator = GeoExtractEvaluator()
all_metrics = evaluator.run_full_evaluation(
    dataset_yaml=dataset_yaml,
    yolo_weights=best_yolo_weights,
    pipeline=pipeline,
)

---
## 6 · Agentic Inference — Demo

Run the dual-model pipeline on sample images.

In [ ]:
import json
from pathlib import Path

# ── Single image analysis ──
# Pick a sample image from the validation set
val_images = list((Path(CFG.data.root) / 'train').rglob('*.tif'))[:3]

for img_path in val_images:
    print(f'\n{"="*60}')
    print(f'Analyzing: {img_path.name}')
    print('='*60)
    
    result = pipeline.analyze(img_path)
    
    print(f'Buildings detected: {result["detection"]["building_count"]}')
    print(f'Density class: {result["context"]["density_class"]}')
    print(f'Processing time: {result["processing_time_s"]}s')
    print(f'\nVLM Analysis:')
    print(result['analysis']['response'][:500])

In [ ]:
# ── Multi-turn conversation demo ──
if val_images:
    img = val_images[0]
    history = []
    
    # Turn 1
    r1 = pipeline.chat(img, history, 'How many buildings are in this image?')
    print(f'Q1: How many buildings?')
    print(f'A1: {r1["response"][:300]}\n')
    history = r1['conversation_history']
    
    # Turn 2 — follow-up
    r2 = pipeline.chat(img, history, 'What urban planning risks do you see?')
    print(f'Q2: What urban planning risks?')
    print(f'A2: {r2["response"][:300]}\n')
    history = r2['conversation_history']
    
    # Turn 3 — deeper follow-up
    r3 = pipeline.chat(img, history, 'Recommend specific interventions to improve livability.')
    print(f'Q3: Recommend interventions')
    print(f'A3: {r3["response"][:300]}')

---
## 7 · Export for Deployment

Export trained weights for FastAPI backend integration.

In [ ]:
from config import EXPORT_DIR
import shutil

# ── Export YOLO to ONNX ──
from yolo_trainer import YOLOTrainer
yt = YOLOTrainer(dataset_yaml=dataset_yaml)
onnx_path = yt.export_for_deployment(format='onnx')
shutil.copy2(onnx_path, EXPORT_DIR / 'yolo_building_detector.onnx')

# ── Copy VLM adapter ──
vlm_export_dir = EXPORT_DIR / 'vlm_adapter'
vlm_export_dir.mkdir(exist_ok=True)
adapter_src = CFG.vlm.output_dir / 'final_adapter'
if adapter_src.exists():
    for f in adapter_src.iterdir():
        shutil.copy2(f, vlm_export_dir / f.name)

print(f'\n✓ Exports ready at {EXPORT_DIR}:')
for f in sorted(EXPORT_DIR.rglob('*')):
    if f.is_file():
        size_mb = f.stat().st_size / 1e6
        print(f'  {f.relative_to(EXPORT_DIR)} ({size_mb:.1f} MB)')

In [ ]:
# Cleanup
pipeline.cleanup()
free_vram()
print('\n🎉 GeoExtract v2 pipeline complete!')